# Bayesian Analysis of Factors Affecting Student Performance
Dataset : https://data.mendeley.com/datasets/5b82ytz489/1
Note: The dataset column name "Overall" is the output variable CGPA


# Install cmdstanpy

In [ ]:
!pip3 install cmdstanpy
import cmdstanpy
cmdstanpy.install_cmdstan()
from cmdstanpy import install_cmdstan
install_cmdstan()


# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import cmdstanpy
import arviz as az
from patsy import dmatrix
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Visualization  

In [ ]:
# Load data & Visualize key predictors

data = pd.read_csv('data.csv')

# Department name abbreviations
dept_mapping = {
    'Computer Science and Engineering': 'CSE',
    'Electrical and Electronic Engineering': 'EEE',
    'Business Administration': 'BA',
    'Economics': 'ECO',
    'Sociology': 'SOC',
    'Law and Human Rights': 'LAW',
    'Public Health': 'PH',
    'Journalism, Communication and Media Studies': 'JOURN',
    'Political Science': 'POLSCI',
    'English': 'ENG'
}
data['Department'] = data['Department'].map(dept_mapping).fillna(data['Department'])

# Define your feature categories
features = ["Preparation", "Last", "Gender", "Computer", "Gaming", "Job", "English", "Extra"]
target = "Overall"

nominal_cols = ['Gender', 'Job', 'Extra', 'Department']  # Added Department
ordinal_cols = ['Computer', 'Preparation', 'Gaming', 'English']
numeric_cols = ['Last']

# Set up the subplot grid - adding 1 for the department count plot
n_cols = 3
n_rows = ((len(features) + 1) + n_cols - 1) // n_cols  # +1 for department plot
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
axes = axes.flatten()

# Create a color palette
nominal_palette = sns.color_palette("Set2")
ordinal_palette = sns.color_palette("viridis")
numeric_palette = sns.color_palette("rocket")

# First plot - Department Count
ax = axes[0]
dept_counts = data['Department'].value_counts().sort_values(ascending=False)

# Create bar plot with different colors
bars = ax.bar(dept_counts.index, dept_counts.values,
              color=sns.color_palette("husl", len(dept_counts)))

# Add grid lines
ax.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.7)
ax.grid(True, which='minor', linestyle=':', linewidth=0.5, alpha=0.5)
ax.minorticks_on()

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height}',
            ha='center', va='bottom')

ax.set_title("Number of Students per Department", pad=20)
ax.set_ylabel("Count")
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

# Remaining feature plots

for i, feature in enumerate(features, start=1):  # Start at 1 to leave space for dept plot
    ax = axes[i]

    # Add grid lines
    ax.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.7)
    ax.grid(True, which='minor', linestyle=':', linewidth=0.5, alpha=0.5)
    ax.minorticks_on()

    if feature in nominal_cols:
        unique_values = data[feature].nunique()
        palette = nominal_palette[:unique_values] if unique_values <= len(nominal_palette) else nominal_palette
        sns.boxplot(data=data, x=feature, y=target, ax=ax, palette=palette)
        ax.set_title(f"CGPA vs {feature}")

    elif feature in ordinal_cols:
        unique_values = data[feature].nunique()
        palette = sns.color_palette("viridis", unique_values)
        sns.boxplot(data=data, x=feature, y=target, ax=ax, palette=palette, order=sorted(data[feature].unique()))
        ax.set_title(f"CGPA vs {feature}")

    elif feature in numeric_cols:
        sns.scatterplot(data=data, x=feature, y=target, ax=ax, palette=numeric_palette,
                        hue=feature,size=feature, sizes=(20, 200))
        ax.set_title(f"CGPA vs {feature}")
        ax.legend().remove()

    if feature in nominal_cols or feature in ordinal_cols:
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

# Hide any unused axes
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout(pad=0.5, h_pad=0.3, w_pad=0.2)

plt.tight_layout()
plt.show()
# plt.savefig("All_visuals.pdf", dpi = 300) # To save the plot in pdf format comment out plt.show(), otherwise it won't be saved

In [ ]:
# #Visualizing nominal data: Department, Gender, Hometown, Job, Extra

# drop insignificant department column

data = data.drop(columns=["Department"])

# separating different column types
nominal_cols = ['Gender', 'Job', 'Extra', 'Hometown']
ordinal_cols = ['Income', 'Computer', 'Preparation', 'Gaming', 'Attendance', 'English', 'Semester']
numeric_cols = ['HSC', 'SSC', 'Last', 'Overall']

# Strip whitespace
data['Income'] = data['Income'].str.strip()

# plt.figure(figsize = (35,10))
# plt.title("Number of students by department")
# plt.bar(data["Department"].value_counts().index, data["Department"].value_counts())
# plt.show()

for feature in nominal_cols:
    if feature!= "Department":
        plt.title(feature)
        plt.pie(data[feature].value_counts(), labels = data[feature].value_counts().index.to_list(), autopct = "%.2f")
        plt.legend()
        plt.show()

#Visualizing ordinal data: Income, Computer, Preparation, Gaming, Attendance, English, Semester

for feature in ordinal_cols:
    plt.title("Histogram of "+feature)
    plt.ylabel("Count")
    plt.bar(data[feature].value_counts().index.to_list(), data[feature].value_counts())
    plt.show()

#Visualizing numeric data: HSC, SSC, Last, Overall

for feature in numeric_cols:
    plt.title("Histogram of "+feature)
    plt.ylabel("Count")
    plt.hist(data[feature])
    plt.show()

#CGPA plotted against each feature
for col in data.columns:
    if col!= "Overall":
        plt.title("CGPA vs "+ col)
        plt.xlabel(col)
        plt.ylabel("CGPA")
        plt.scatter(data[col],data["Overall"], s = 5)
        plt.show()

# Data Preprocessing

In [ ]:
# One-hot encode nominal categorical variables
data = pd.get_dummies(data, columns = nominal_cols, drop_first = True)

# Mapping for the ordinal variables: Income, Preparation, Gaming, Semester, Attendance
income_mapping = {
    "Low (Below 15,000)": 1,
    "Lower middle (15,000-30,000)": 2,
    "Upper middle (30,000-50,000)": 3,
    "High (Above 50,000)": 4
}
data['Income'] = data['Income'].map(income_mapping)

# Mapping study preparation time and gaming time to numeric values.
prep_mapping = {
    "0-1 Hour": 1,
    "2-3 Hours": 2,
    "More than 3 Hours": 3
}
data['Preparation'] = data['Preparation'].map(prep_mapping)
data['Gaming'] = data['Gaming'].map(prep_mapping)

# Convert strings in Semester column like "6th", "7th", etc. to integers.
data['Semester'] = data['Semester'].str.extract('(\d+)').astype(int)

# Mapping attendance ranges to numeric values
attendance_mapping = {
    "80%-100%": 4,
    "60%-79%": 3,
    "40%-59%": 2,
    "Below 40%": 1
}

# Apply the mapping to the Attendance column.
data['Attendance'] = data['Attendance'].str.strip().map(attendance_mapping)

# Use the .replace() method to map True/False to 1/0
data = data.replace({True: 1, False: 0})

# Columns to scale (features only, excluding 'Overall')
cols_to_scale = ['HSC', 'SSC', 'Last']

# Initialize separate scalers
feature_scaler = StandardScaler()  # For features (HSC, SSC, Last)
target_scaler = StandardScaler()   # For target (Overall)

# Scale the features (modifies HSC, SSC, Last in-place)
data[cols_to_scale] = feature_scaler.fit_transform(data[cols_to_scale])

# Scale 'Overall' and add as a new column (without affecting the original)
data['Overall_scaled'] = target_scaler.fit_transform(data[['Overall']])

# Create descriptive statistics table
#Variance
stats_data = data.describe()
variance = pd.DataFrame([data.var()])
variance = variance.rename(index = {0:"variance"})
#median
median = pd.DataFrame([data.median()])
median = median.rename(index = {0:"median"})
#skew
skew = pd.DataFrame([data.skew()])
skew = skew.rename(index = {0:"skew"})
#kurtois
kurtosis = pd.DataFrame([data.kurtosis()])
kurtosis = kurtosis.rename(index = {0:"kurtosis"})

stats = pd.concat([stats_data, variance, median, skew, kurtosis])
stats = stats.drop(columns=["Overall_scaled","Overall"], axis = 1)
stats


# Helper Functions

In [ ]:
# Function to calculate Rsquared values

def Rsquared(y_pred, y_true):

    residual_sum_squares = np.sum((y_true - y_pred)**2)
    total_sum_squares = np.sum((y_true - np.mean(y_true))**2)

    Rsquared = 1- residual_sum_squares/total_sum_squares

    return Rsquared

# Function to plot regression result

def plot_regression(plot_title, data, pred_overall):

    # Scatter plot to compare actual vs predicted GPA
    plt.figure(figsize=(8, 4))
    plt.scatter(data["Overall_scaled"], pred_overall, alpha=0.4, color="blue", label="Predicted Scaled CGPA")
    plt.scatter(data["Overall_scaled"], data["Overall_scaled"], alpha=0.4, color="red", label="Actual Scaled CGPA")  # Plot actual values in red
    plt.xlabel("Actual Scaled CGPA")
    plt.ylabel("Predicted Scaled CGPA")
    plt.title(plot_title)
    plt.grid()
    plt.legend()
    plt.show()

# Model with all factors (no interaction)

In [ ]:
#Dictionary to store the Rsquared values for each model
R_squared_vals = {}

In [ ]:
'''Index(['HSC', 'SSC', 'Income', 'Computer', 'Preparation', 'Gaming',
       'Attendance', 'English', 'Semester', 'Last', 'Overall', 'Gender_Female',
       'Gender_Male', 'Job_No', 'Job_Yes', 'Extra_No', 'Extra_Yes',
       'Hometown_City', 'Hometown_Village', 'Overall_scaled'],
      dtype='object')'''

predictor_cols = [col for col in data.columns if col not in ["Overall_scaled", "Overall"]]
K_value = data.drop(columns=["Overall_scaled", "Overall"]).shape[1]

# Create a dictionary to pass data to Stan
stan_data_all = {
   "N": data.shape[0],
   "K": K_value,
   "X": data.drop(columns=["Overall_scaled", "Overall"]).values,
   "Y": data["Overall_scaled"].values
}

# Stan model code to save in a file

full_model = """
data {
  // data size
  int<lower=0> N;
  int<lower=0> K;

  // target variable
  vector[N] Y;

  // predictors
  matrix[N, K] X;
}

parameters {
    vector[K] beta;      // Regression coefficients
    real alpha;          // Intercept
    real<lower=0> sigma; // Error term
}
model {
  // priors

  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  sigma ~ normal(0,10);

  // model
  // Likelihood function
  Y ~ normal(X*beta + alpha, sigma);
}
"""
# save model in stan file
stan_model_file = "all_factor_regression.stan"
with open(stan_model_file, "w") as f:
    f.write(full_model)

# Compile and run Stan model

stan_full_model = cmdstanpy.CmdStanModel(stan_file = stan_model_file)
fit_full = stan_full_model.sample(data = stan_data_all, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

# Posterior Analysis & Visualization
fit_full.summary().to_csv("full_stan_results.csv")
fit_full.summary()

In [ ]:
full_posterior_samples = fit_full.draws_pd()

# Extract posterior means for coefficients and intercept
full_beta_means = full_posterior_samples[[col for col in full_posterior_samples.columns if "beta" in col]].mean().values
full_alpha_mean = full_posterior_samples["alpha"].mean()

# Compute predicted GPA using the regression formula
pred_overall_full = data.drop(columns=["Overall_scaled", "Overall"]).dot(full_beta_means) + full_alpha_mean

# Compute Rsquare value
R_squared_full = Rsquared(y_pred = pred_overall_full, y_true = data["Overall_scaled"])
print("R squared for model with all factors (no interaction): ", R_squared_full)
R_squared_vals["all_factors"] = round(R_squared_full,3)

# Plot regression line
plot_regression("Full Model Regression Plot", data, pred_overall_full)

In [ ]:
# Check for divergences
divergences = full_posterior_samples['divergent__']
print("Number of divergences:", divergences.sum())

# Academic Model

In [ ]:
academic_cols = ['HSC', 'SSC','Preparation', 'Attendance', 'Semester', 'Last', 'Overall_scaled']
data_academic = data[academic_cols]
K_value = data_academic.drop(columns=["Overall_scaled"]).shape[1]

# Create a dictionary to pass data to Stan
stan_data_academic = {
   "N": data_academic.shape[0],
   "K": K_value,
   "X": data_academic.drop(columns=["Overall_scaled"]).values,
   "Y": data_academic["Overall_scaled"].values
}

# Stan model code to save in a file
# efficient way to write the stan model program

academic_model = """
data {
  // data size
  int<lower=0> N;
  int<lower=0> K;

  // target variable
  vector[N] Y;

  // predictors
  matrix[N, K] X;
}

parameters {
    vector[K] beta;      // Regression coefficients
    real alpha;          // Intercept
    real<lower=0> sigma; // Error term
}
model {
  // priors

  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  sigma ~ normal(0,10);

  // model
  // Likelihood function
  Y ~ normal(X*beta + alpha, sigma);
}
"""

# save model in stan file
stan_model_file = "student_performance_regression.stan"
with open(stan_model_file, "w") as f:
    f.write(academic_model)

# Compile and run Stan model
stan_academic_model = cmdstanpy.CmdStanModel(stan_file = stan_model_file)
fit_academic = stan_academic_model.sample(data = stan_data_academic, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

# Posterior Analysis & Visualization
fit_academic.summary().to_csv("academic_stan_results.csv")
fit_academic.summary()

In [ ]:
# Extract posterior samples
academic_posterior_samples = fit_academic.draws_pd()

# Extract posterior means for coefficients and intercept
academic_beta_means = academic_posterior_samples[[col for col in academic_posterior_samples.columns if "beta" in col]].mean().values
academic_alpha_mean = academic_posterior_samples["alpha"].mean()

# Compute predicted GPA using the regression formula
pred_overall_academic = data_academic.drop(columns=["Overall_scaled"]).dot(academic_beta_means) + academic_alpha_mean

# Compute Rsquare value
R_squared_academic = Rsquared(y_pred = pred_overall_full, y_true = data_academic["Overall_scaled"])
print("R squared for model with academic factors (no interaction): ", R_squared_academic)
R_squared_vals["academic"] = round(R_squared_academic,3)

# Plot regression line
plot_regression("Academic Model Plot", data_academic, pred_overall_academic)


# Academic interactions

In [ ]:
'''academic_cols = [
    'HSC', 'SSC','Preparation', 'Attendance', 'Semester', 'Last', 'Overall', 'Overall_scaled'
    ]''' # Added for easy reference to the columns

# Create interaction terms
X_ac = dmatrix('~ 0 + Preparation + Attendance + Last + Last:Preparation + Preparation:Attendance + Last:Attendance',
             data_academic,
             return_type='dataframe')

# Prepare Stan data for interaction effect
stan_data = {
    "N": X_ac.shape[0],
    "K": X_ac.shape[1],
    "X": X_ac.values,
    "Y": data_academic["Overall_scaled"].values
}

academic_interaction_model = """
data {
  int<lower=0> N;
  int<lower=0> K;
  matrix[N, K] X;
  vector[N] Y;
}
parameters {
  real alpha;
  vector[K] beta;
  real<lower=0> sigma;
}
model {
  alpha ~ normal(0, 10);
  beta ~ normal(0, 10);
  sigma ~ normal(0, 10);

  Y ~ normal(alpha + X * beta, sigma);
}
"""
# save model in stan file
stan_model_file = 'interaction_effects_academic_model.stan'
with open(stan_model_file, "w") as f:
    f.write(academic_interaction_model)
stan_model = cmdstanpy.CmdStanModel(stan_file=stan_model_file)
fit_academic_inter = stan_model.sample(data=stan_data, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

# Posterior Analysis & Visualization
fit_academic_inter.summary().to_csv("academic_interaction_stan_results.csv")
fit_academic_inter.summary()

In [ ]:
# Extract posterior samples
academic_posterior_samples_inter = fit_academic_inter.draws_pd()

# Extract posterior means for coefficients and intercept
academic_beta_means_inter = academic_posterior_samples_inter[[col for col in academic_posterior_samples_inter.columns if "beta" in col]].mean().values
academic_alpha_mean_inter = academic_posterior_samples_inter["alpha"].mean()

# Compute predicted GPA using the regression formula
# pred_overall_academic_inter = data_academic.drop(columns=["Overall_scaled"]).dot(academic_beta_means_inter) + academic_alpha_mean_inter
pred_overall_academic_inter = X_ac.dot(academic_beta_means_inter) + academic_alpha_mean_inter

# Compute Rsquare value
R_squared_academic_inter = Rsquared(y_pred = pred_overall_academic_inter, y_true = data_academic["Overall_scaled"])
print("R squared for academic interactive model: ", R_squared_academic_inter)
R_squared_vals["academic_inter"] = round(R_squared_academic_inter,3)

# Plot regression line
plot_regression("Academic Interaction Model Regression Plot", data_academic, pred_overall_academic_inter)

# Non-Academic Model

In [ ]:
non_academic_cols=['Gender_Male','Income','Hometown_Village','Computer','Gaming','Job_Yes','English','Extra_Yes','Overall_scaled'] #consider the features after one hot encoding
data_non_academic = data[non_academic_cols]

K_value = data_non_academic.drop(columns=["Overall_scaled"]).shape[1]

# Create a dictionary to pass data to Stan
stan_data_main_effects = {
   "N": data_non_academic.shape[0],
   "K": K_value,
   "X": data_non_academic.drop(columns=["Overall_scaled"]).values,
   "Y": data_non_academic["Overall_scaled"].values
}

# Stan file code
non_academic_model = """
data {
  // data size
  int<lower=0> N;
  int<lower=0> K;

  // target variable
  vector[N] Y;

  // predictors
  matrix[N, K] X;
}

parameters {
    vector[K] beta;      // Regression coefficients
    real alpha;          // Intercept
    real<lower=0> sigma; // Error term
}
model {
  // priors

  alpha ~ normal(0, 10);         // Intercept centered around 3
  beta ~ normal(0, 10);          // Effects expected to be modest
  sigma ~ normal(0, 10);       // Supports small sigma values

  // model
  // Likelihood function
  Y ~ normal(X*beta + alpha, sigma);
}
"""

# save model in stan file
stan_model_file = "non_academic_student_performance_regression.stan"
with open(stan_model_file, "w") as f:
    f.write(non_academic_model)

stan_non_academic_model = cmdstanpy.CmdStanModel(stan_file = stan_model_file)
fit_non_academic = stan_non_academic_model.sample(data = stan_data_main_effects, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

# Posterior Analysis & Visualization
fit_non_academic.summary().to_csv("non_academic_stan_results.csv")
fit_non_academic.summary()

In [ ]:
# Extract posterior samples
non_academic_posterior_samples = fit_non_academic.draws_pd()

# Extract posterior means for coefficients and intercept
non_academic_beta_means = non_academic_posterior_samples[[col for col in non_academic_posterior_samples.columns if "beta" in col]].mean().values
non_academic_alpha_mean = non_academic_posterior_samples["alpha"].mean()

# Compute predicted GPA using the regression formula
pred_overall_non_academic = data_non_academic.drop(columns=["Overall_scaled"]).dot(non_academic_beta_means) + non_academic_alpha_mean

# Compute Rsquare value
R_squared_non_academic = Rsquared(y_pred = pred_overall_non_academic, y_true = data_non_academic["Overall_scaled"])
print("R squared for model with non-academic factors (no interactions): ", R_squared_non_academic)
R_squared_vals["non_academic"] = round(R_squared_non_academic,3)

# Plot regression line
plot_regression("Non-Academic Model Regression Plot", data_non_academic, pred_overall_non_academic)

# Non-academic Interactions

In [ ]:
# Create interaction terms
X_nac = dmatrix('~ 0 + Gender_Male + Computer + Gaming + Extra_Yes + English + Gender_Male:Gaming + Computer:Gaming + Gaming:Extra_Yes + Computer:Extra_Yes + Gender_Male:Extra_Yes + Gender_Male:English',
             data_non_academic,
             return_type='dataframe')

# Prepare Stan data for interaction effect
stan_data = {
    "N": X_nac.shape[0],
    "K": X_nac.shape[1],
    "X": X_nac.values,
    "Y": data_non_academic["Overall_scaled"].values
}

non_academic_interaction_model = """
data {
  int<lower=0> N;
  int<lower=0> K;
  matrix[N, K] X;
  vector[N] Y;
}
parameters {
  real alpha;
  vector[K] beta;
  real<lower=0> sigma;
}
model {
  alpha ~ normal(0,10);
  beta ~ normal(0, 10);
  sigma ~ normal(0,10);

  Y ~ normal(alpha + X * beta, sigma);
}
"""
# save model in stan file
stan_model_file = 'interaction_effects_model.stan'
with open(stan_model_file, "w") as f:
    f.write(non_academic_interaction_model)
stan_model = cmdstanpy.CmdStanModel(stan_file=stan_model_file)
fit_non_acad_inter = stan_model.sample(data=stan_data, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

# Posterior Analysis & Visualization
fit_non_acad_inter.summary().to_csv("non_academic_interaction_stan_results.csv")
fit_non_acad_inter.summary()

In [ ]:
# Extract posterior samples
non_acad_posterior_samples_inter = fit_non_acad_inter.draws_pd()

# Extract posterior means for coefficients and intercept
non_acad_beta_means_inter = non_acad_posterior_samples_inter[[col for col in non_acad_posterior_samples_inter.columns if "beta" in col]].mean().values
non_acad_alpha_mean_inter = non_acad_posterior_samples_inter["alpha"].mean()

# Compute predicted GPA using the regression formula
# pred_overall_non_acad_inter = data_non_academic.drop(columns=["Overall_scaled"]).dot(non_acad_beta_means_inter) + non_acad_alpha_mean_inter
pred_overall_non_acad_inter = X_nac.dot(non_acad_beta_means_inter) + non_acad_alpha_mean_inter

# Compute Rsquare value
R_squared_non_academic_inter = Rsquared(y_pred = pred_overall_non_acad_inter, y_true = data_non_academic["Overall_scaled"])
print("R squared for model with non-academic factors (with interactions): ", R_squared_non_academic_inter)
R_squared_vals["non_academic_inter"] = round(R_squared_non_academic_inter,3)

# Plot regression line
plot_regression("Non-Academic Interaction Model Regression Plot", data_non_academic, pred_overall_non_acad_inter)

# Interactive sequence Models

In [ ]:
def stan_fit(X, data):

    # Prepare Stan data for interaction effect
    stan_data = {
        "N": X.shape[0],
        "K": X.shape[1],
        "X": X.values,
        "Y": data["Overall_scaled"].values
    }

    academic_non_academic_interaction_model = """
    data {
      int<lower=0> N;
      int<lower=0> K;
      matrix[N, K] X;
      vector[N] Y;
    }
    parameters {
      real alpha;
      vector[K] beta;
      real<lower=0> sigma;
    }
    model {
      alpha ~ normal(0, 10);
      beta ~ normal(0, 10);
      sigma ~ normal(0, 10);

      Y ~ normal(alpha + X * beta, sigma);
    }
    """
    # save model in stan file
    stan_model_file = 'academic_nonacademic_interaction_effects_model.stan'

    with open(stan_model_file, "w") as f:
        f.write(academic_non_academic_interaction_model)

    stan_model = cmdstanpy.CmdStanModel(stan_file=stan_model_file)
    fit = stan_model.sample(data=stan_data, chains=4, iter_sampling=2000, iter_warmup=1000, seed=42)

    # Posterior Analysis & Visualization
    #fit.summary().to_csv("academic_non_academic_interaction_stan_results.csv")
    fit.summary()

    return fit

In [ ]:
from patsy import dmatrix

X_1 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes',data, return_type='dataframe')
X_2 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male',data, return_type='dataframe')
X_3 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer',data, return_type='dataframe')
X_4 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English',data, return_type='dataframe')
X_5 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English + Last:Gaming',data, return_type='dataframe')
X_6 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English + Last:Gaming +  Last:Gender_Male',data, return_type='dataframe')
X_7 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English + Last:Gaming +  Last:Gender_Male + Last:Extra_Yes',data, return_type='dataframe')
X_8 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English + Last:Gaming +  Last:Gender_Male + Last:Extra_Yes + Last:English',data, return_type='dataframe')
X_9 = dmatrix('~ 0 + Preparation + Last + Gender_Male + Computer + English + Gaming + Extra_Yes + Preparation:Gender_Male + Preparation:Computer + Preparation:English + Last:Gaming +  Last:Gender_Male + Last:Extra_Yes + Last:English + Last:Computer',data, return_type='dataframe')

models = {"m1": X_1, "m2": X_2, "m3": X_3, "m4":X_4, "m5":X_5, "m6":X_6, "m7":X_7, "m8":X_8, "m9":X_9}
model_fits = []
m_regression_coeffs = {}

for model_name, model_X in models.items():
    fit = stan_fit(X = model_X, data = data)

    model_fits.append(fit)

    posterior_samples = fit.draws_pd()       #posterior samples

    # Extract posterior means for coefficients and intercept
    beta_means = posterior_samples[[col for col in posterior_samples.columns if "beta" in col]].mean().values
    alpha_mean = posterior_samples["alpha"].mean()

    m_regression_coeffs[model_name] = [np.round(alpha_mean,3), np.round(beta_means,3)]


    # Compute predicted GPA using the regression formula
    pred_overall = model_X.dot(beta_means) + alpha_mean

    if model_name == "m6":
        plot_regression("Academic:Non-Academic Model-6 Regression Plot", data = data, pred_overall = pred_overall)
    else:
        plot_regression("Model-"+model_name+" Regression Plot", data = data, pred_overall = pred_overall)

    R_squared = Rsquared(y_pred = pred_overall, y_true = data["Overall_scaled"])

    R_squared_vals[model_name] = round(R_squared,3)

    print("R squared: ", R_squared)


In [ ]:
i = 0
for fit in model_fits:
    i = i+1
    print("Fit summary for model", i)
    print(fit.summary())
    fit.summary().to_csv("sequential_stan_results"+str(i)+".csv")

# Comparing Models using AIC/BIC and Leave One out Cross Validation

In [ ]:
def compute_log_likelihood(y_true, y_pred, sigma, return_individual=False):
    # Compute per-observation log-likelihood
    individual_ll = -0.5 * np.log(2 * np.pi * sigma**2) - 0.5 * ((y_true - y_pred)**2 / sigma**2)
    if return_individual:
        return individual_ll
    else:
        return np.sum(individual_ll)

def compute_aic_bic(stan_fit, data, y_col, X_cols):
    # Extract posterior samples as a DataFrame.
    posterior_samples = stan_fit.draws_pd()

    # Compute posterior means.
    alpha_mean = posterior_samples["alpha"].mean()
    sigma_mean = posterior_samples["sigma"].mean()
    beta_cols = [c for c in posterior_samples.columns if c.startswith("beta")]
    beta_means = posterior_samples[beta_cols].mean().values

    # Prepare data.
    X = data[X_cols].values  # shape (n_obs, n_predictors)
    y = data[y_col].values   # shape (n_obs,)

    # Compute predicted values using the posterior means.
    y_pred = alpha_mean + X.dot(beta_means)

    # Compute the total log-likelihood.
    total_log_lik = compute_log_likelihood(y, y_pred, sigma_mean, return_individual=False)

    n = len(y)
    # Number of parameters: beta's + alpha + sigma.
    k = len(beta_means) + 2

    # AIC and BIC formulas.
    aic = 2 * k - 2 * total_log_lik
    bic = k * np.log(n) - 2 * total_log_lik
    return aic, bic

def compute_aic_bic_dmatrix(stan_fit, X_design, y):
    # Extract posterior samples as a DataFrame.
    posterior_samples = stan_fit.draws_pd()

    # Compute posterior means.
    alpha_mean = posterior_samples["alpha"].mean()
    sigma_mean = posterior_samples["sigma"].mean()

    # Identify beta columns (assuming they are named 'beta[1]', 'beta[2]', etc.)
    beta_cols = [c for c in posterior_samples.columns if c.startswith("beta")]
    beta_means = posterior_samples[beta_cols].mean().values

    # Compute predicted values using the posterior means.
    y_pred = alpha_mean + X_design.dot(beta_means)

    # Compute the total log-likelihood.
    total_log_lik = compute_log_likelihood(y, y_pred, sigma_mean, return_individual=False)

    n = len(y)
    # Number of parameters: beta's + alpha + sigma.
    k = len(beta_means) + 2

    # AIC and BIC formulas.
    aic = 2 * k - 2 * total_log_lik
    bic = k * np.log(n) - 2 * total_log_lik
    return aic, bic

def compute_log_lik_array(fit, data, y_col, X_cols):
    posterior_samples = fit.draws_pd()
    n_draws = len(posterior_samples)

    # Prepare data.
    X = data[X_cols].values
    y = data[y_col].values
    n_obs = len(y)

    beta_cols = [col for col in posterior_samples.columns if col.startswith("beta")]
    log_lik_array = np.empty((n_draws, n_obs))

    # Loop over each posterior draw.
    for i, row in posterior_samples.iterrows():
        alpha = row["alpha"]
        sigma = row["sigma"]
        beta = row[beta_cols].values
        y_pred = alpha + X.dot(beta)
        # Get per-observation log-likelihood.
        log_lik_array[i, :] = compute_log_likelihood(y, y_pred, sigma, return_individual=True)

    # If chain information is available, reshape accordingly.
    try:
        n_chains = int(fit.runset.chains)
        n_samples = n_draws // n_chains
        log_lik_array = log_lik_array.reshape((n_chains, n_samples, n_obs))
    except Exception:
        # If chain info is not available, leave it as (n_draws, n_obs)
        pass

    return log_lik_array

def get_inference_data(fit, data, y_col, X_cols):
    posterior_samples = fit.draws_pd().to_dict(orient="list")
    log_lik_array = compute_log_lik_array(fit, data, y_col, X_cols)
    idata = az.from_dict(
        posterior=posterior_samples,
        log_likelihood={"log_lik": log_lik_array}
    )
    return idata


In [ ]:
#Full
full_aic, full_bic = compute_aic_bic(
    stan_fit = fit_full,
    data     = data,
    y_col    = "Overall_scaled",
    X_cols   = [col for col in data.columns
                if col not in ( "Overall_scaled")]
)

print("Full AIC:", full_aic)
print("Full BIC:", full_bic)
#Academic
academic_aic, academic_bic = compute_aic_bic(
    stan_fit = fit_academic,
    data     = data_academic,
    y_col    = "Overall_scaled",
    X_cols   = [col for col in data_academic.columns
                if col not in ( "Overall_scaled")]
)
print("Academic AIC:", academic_aic)
print("Academic BIC:", academic_bic)
#Academic Inter
academic_inter_aic, academic_inter_bic = compute_aic_bic_dmatrix(fit_academic_inter, X_design=X_ac, y=data_academic["Overall_scaled"])

print("Academic-interaction AIC:", academic_inter_aic)
print("Academic-Interaction BIC:", academic_inter_bic)
#non-Acdemic
non_academic_aic, non_academic_bic = compute_aic_bic(
    stan_fit = fit_non_academic,
    data     = data_non_academic,
    y_col    = "Overall_scaled",
    X_cols   = [col for col in data_non_academic.columns
                if col not in ( "Overall_scaled")]
)
print("Non-Academic AIC:", non_academic_aic)
print("Non-Academic BIC:", non_academic_bic)

#Non academic interaction
non_academic_inter_aic, non_academic_inter_bic = compute_aic_bic_dmatrix(fit_non_acad_inter, X_design=X_nac, y=data_non_academic["Overall_scaled"])

print("Non-Academic inter AIC:", non_academic_inter_aic)
print("Non-Academic inter BIC:", non_academic_inter_bic)
'''
X_cols = [col for col in data.columns if col not in ("Overall_scaled", "Intercept")]
X = data[X_cols]
#fit = stan_fit(X, data)
ac_nonac_interactive_aic, ac_nonac_interactive_bic = compute_aic_bic(
    stan_fit=fit,
    data=data,
    y_col="Overall_scaled",
    X_cols=X_cols
)

print("Academic:Non-Academic AIC:", ac_nonac_interactive_aic)
print("Academic:Non-Academic BIC:", ac_nonac_interactive_bic)
'''
#best
fit_m6 = model_fits[list(models.keys()).index("m6")]
aic, bic = compute_aic_bic_dmatrix(fit_m6, X_design=X_6, y=data["Overall_scaled"])
print("Best Academic:Non-Academic AIC:", aic)
print("Best Academic:Non-Academic BIC:", bic)


In [ ]:
#For Full
data_full = data[predictor_cols].copy()
data_full["Overall_scaled"] = data["Overall_scaled"]

idata_full = get_inference_data(
    fit=fit_full,
    data=data_full,
    y_col="Overall_scaled",
    X_cols=predictor_cols
)
loo_full = az.loo(idata_full)
print("LOO for Full Model:", loo_full)

# Academic Model
idata_academic = get_inference_data(
    fit=fit_academic,
    data=data_academic,
    y_col="Overall_scaled",
    X_cols=[col for col in data_academic.columns if col != "Overall_scaled"]
)
loo_academic = az.loo(idata_academic)
print("LOO for Academic Model:", loo_academic)

# academic interaction Model
# Create a DataFrame that contains the predictors (interaction terms) and the response.
academic_cols = X_ac.columns.tolist()
data_academic_interaction = X_ac.copy()
data_academic_interaction["Overall_scaled"] = data_academic["Overall_scaled"]
idata_academic_inter = get_inference_data(
    fit=fit_academic_inter,
    data=data_academic_interaction,
    y_col="Overall_scaled",
    X_cols=academic_cols
)
loo_academic_inter = az.loo(idata_academic_inter)
print("LOO for Academic interaction Model:", loo_academic_inter)

# non-academic
idata_non_academic = get_inference_data(
    fit=fit_non_academic,
    data=data_non_academic,
    y_col="Overall_scaled",
    X_cols=[col for col in data_non_academic.columns if col != "Overall_scaled"]
)
loo_non_academic = az.loo(idata_non_academic)
print("LOO for Non-Academic Model:", loo_non_academic)

# Non-Academic interaction Model
non_academic_cols = X_nac.columns.tolist()
data_non_academic_interaction = X_nac.copy()
data_non_academic_interaction["Overall_scaled"] = data_non_academic["Overall_scaled"]

idata_non_academic_inter = get_inference_data(
    fit=fit_non_acad_inter,
    data=data_non_academic_interaction,
    y_col="Overall_scaled",
    X_cols=non_academic_cols #extract list from dataframe
)
loo_non_academic_inter = az.loo(idata_non_academic_inter)
print("LOO for Non-Academic interaction Model:", loo_non_academic_inter)

#Best Academic and Non-academic

X_m6 = models["m6"]
academic_non_academic_cols = X_m6.columns.tolist()
data_academic_non_academic_interaction = X_m6.copy()
data_academic_non_academic_interaction["Overall_scaled"] = data["Overall_scaled"]
fit_m6 = model_fits[list(models.keys()).index("m6")]
idata_academic_non_academic_inter = get_inference_data(
    fit=fit_m6,
    data=data_academic_non_academic_interaction,
    y_col="Overall_scaled",
    X_cols=academic_non_academic_cols
)
loo_academic_non_academic_inter = az.loo(idata_academic_non_academic_inter)
print("LOO for Academic and Non-Academic interaction Model (m6):", loo_academic_non_academic_inter)


In [ ]:

model_dict = {
    "Model with all factors":idata_full,
    "Academic Model ":idata_academic,
    "Non-Academic Model":idata_non_academic,
    "Academic Interaction Model": idata_academic_inter,
    "Non Academic Interaction": idata_non_academic_inter,
    "Academic and Non-Academic Interaction": idata_academic_non_academic_inter
}

comparison = az.compare(model_dict, ic="loo")
print(comparison)


# Result Summaries

In [ ]:
#Compare Model R squared values
print("R^2 values for each model:")
df_Rsquared = pd.DataFrame(R_squared_vals, index = [0])
df_Rsquared

In [ ]:
# #Table for interactive sequence models
m_regression_coeffs

In [ ]:
df_regression_coeffs = pd.DataFrame(m_regression_coeffs, index = ["alpha", "betas"])
df_regression_coeffs